In [10]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [11]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_52181/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [13]:
# 测试下to_csv函数
all_data.iloc[:10].to_csv('all_data_10.csv')
all_data_10 = pd.read_csv('all_data_10.csv')
all_data_10[['cv_id', 'eduTracks']]

,cv_id,eduTracks
0,2c9207157bf91042017c0d2beef00470,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":..."
1,2c9207157bf91042017c0d979cbb047d,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":..."
2,2c9207157bf91042017c7232fbab3be1,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":..."
3,2c9207157d308592017d4a94cb1113a1,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":..."
4,2c9207157dabb0a3017dbca16cdf4c26,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":..."
5,2c9207157dbedf8d017dc1503c891402,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":..."
6,2c9207157dbedf8d017dc1503c891402,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":..."
7,2c9207157dbedf8d017dc651bdcf5638,"['{""degree"": ""本科"", ""endDate"": ""2017-01"", ""id"":..."
8,2c9207157dbedf8d017dc651bdcf5638,"['{""degree"": ""本科"", ""endDate"": ""2017-01"", ""id"":..."
9,2c9207157dbedf8d017dc7561a247ced,"['{""degree"": ""大专"", ""endDate"": ""2015-06"", ""id"":..."


Index(['Unnamed: 0', 'cv_id', 'jd_id', 'jd_code', 'title', 'deadline',
       'category_id', 'category_name', 'company_id', 'company_name',
       'recruit_number', 'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'mainta

In [5]:
all_data[['work_age', 'workYear']]

,work_age,workYear
0,5-10年,13.0
1,3-5年,7.0
2,10年以上,14.0
3,3-5年,3.0
4,3-5年,5.0
...,...,...
81291,5-10年,1.0
81292,10年以上,23.0
81293,10年以上,23.0
81294,3-5年,5.0


## 理清需要用的特征


In [9]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 'pay_months', 'currentSalary', 'desiredSalary']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_words', 'equal_job']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [11]:
all_data[['cv_id', 'jd_id']+raw_columns]

,cv_id,jd_id,age,recruit_number,workYear,min_annual_salary,max_annual_salary,pay_months,currentSalary,desiredSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,80.0,150.0,NaN,NaN,NaN
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,28.0,35.0,NaN,NaN,NaN
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,25.0,35.0,NaN,NaN,NaN
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,60.0,500.0,NaN,NaN,NaN
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,NaN,NaN,12.0,保密,25 - 30k · 15薪
...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,41.0,1,1.0,NaN,NaN,12.0,6000,6千-8千/月
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,47.0,1,23.0,NaN,NaN,12.0,60+期权,100
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,47.0,1,23.0,NaN,NaN,13.0,60+期权,100
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,30.0,30,5.0,NaN,NaN,14.0,25K,28K


## tfidf_pca

In [12]:
# 将这个特征所需的列准备好
# 不过似乎没必要，都是拿all_data
position_columns = ['cv_id', 'jd_id', 'currentPosition', 'desiredPosition']
all_data[position_columns]

,cv_id,jd_id,currentPosition,desiredPosition
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,副总裁/副总经理,NaN
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,数字化营销经理,NaN
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,NaN
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,泰国国家经理,NaN
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,项目经理,项目经理/主管
...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,生产工、包装工,生产制造/机械设计/制造/机械设备工程师
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,高级市场总监,市场/市场/营销/市场营销
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,高级市场总监,市场/市场/营销/市场营销
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,java,技术/后端开发/Java


In [1]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

'''
1.不论是单列文本还是多列文本都可以通过这个函数统一获取tfidf_pca向量
2.给apply函数传入了参数，免去了global申明
'''

def load_csv_data(data_path):
    '''
    读取csv文件
    '''
    df = pd.read_csv(data_path)
    return df

def col_jieba_fun(series, col_name):
    '''
    将文本字符串切词成列表
    '''
    col = series[col_name]
    #print(col)
    # 加入特例判断 *Tracks。'[{},{}]', json无法解析。
    if col_name.endswith("Tracks"):
        col_list = jieba.lcut(col, cut_all=False)
        return col_list

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 列表变字符串
    # 对于中文，进入jieba前不需要添加空格；不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)

    # 切词
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

def col_jieba_filter_fun(series, col_name_jieba):
    '''
    对切词后的列表进行过滤
    '''
    col_list_filter = []
    
    # 得到切词后的文本列表
    col_list = series[col_name_jieba]

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

def get_tfidf(df, col_name):
    '''
    将文本列转成tfidf向量
    '''
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

def get_tfidf_pca(tfidf, n=20):
    '''
    将tfidf向量降维
    '''
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

def col_merge_fun(series, col_name_jieba_filter_list):
    '''
    合并多个文本列
    '''
    merge = ''
    for col in col_name_jieba_filter_list:
        merge = merge + series[col] + ' '
    return merge.strip(' ')

def get_tfidf_pca_from_text_cols(data_path, col_name_list, dimension):
    '''
    从多个文本列计算tfidf_pca

    :param data_path csv数据路径
    :param col_name_list 文本列列名列表
    :param dimension tfidf经过pca降维后的维度
    :returns: tfidf_pca向量
    '''
    # 读取csv文件
    df = load_csv_data(data_path)

    # 存储经过分词和过滤后的列名
    col_name_jieba_filter_list = []

    for col_name in col_name_list:

        col_name_jieba = col_name + '_jieba'
        col_name_jieba_filter = col_name_jieba + '_filter'
        col_name_jieba_filter_list.append(col_name_jieba_filter)

        # step1 空值填充
        df[col_name].fillna('', inplace=True)

        # step2 jieba分词
        df[col_name_jieba] = df.apply(col_jieba_fun, axis=1, args=(col_name, ))

        # step3 分词过滤
        df[col_name_jieba_filter] = df.apply(col_jieba_filter_fun, axis=1, args=(col_name_jieba, ))

        print("\n=================================={}==================================".format(col_name))
        print(df[[col_name, col_name_jieba, col_name_jieba_filter]])

    print(col_name_jieba_filter_list)
    
    merge_col_jieba_filter = "_".join(col_name_list) + '_jieba_filter'
    df[merge_col_jieba_filter] = df.apply(col_merge_fun, axis=1, args=(col_name_jieba_filter_list, ))

    print("\n=================================={}==================================".format('以上各列分词过滤后合并的新列'))
    print(df[[merge_col_jieba_filter]])

    # step4 得到tfidf
    tfidf, vectorizer = get_tfidf(df, merge_col_jieba_filter)
    print("\n=================================={}==================================".format('tfidf向量'))
    print(tfidf)

    # step5 得到tfidf_pca
    tfidf_pca = get_tfidf_pca(tfidf, dimension)
    print("\n=================================={}==================================".format('tfidf_pca向量'))
    print(tfidf_pca)

    return tfidf_pca


if __name__ == "__main__":
    print("running...")

    data_path = '../data/all_sample_20220821_spark.csv'
    dimension = 20
    
    print("\n从文本列获取tfidf_pca向量\n")
    col_name_list1 = ['title', 'category_name', 'tags']
    col_name_list2 = ['description']
    col_name_list3 = ['requirement']

    col_name_list4 = ['currentPosition', 'desiredPosition']
    col_name_list5 = ['skills']
    col_name_list6 = ['jobTracks']
   
    tfidf_pca1 = get_tfidf_pca_from_text_cols(data_path, col_name_list1, dimension=30)
    tfidf_pca1.to_csv('title_category_tags_tfidf_pca.csv')

    tfidf_pca2 = get_tfidf_pca_from_text_cols(data_path, col_name_list2, dimension=70)
    tfidf_pca2.to_csv('description_tfidf_pca.csv')

    tfidf_pca3 = get_tfidf_pca_from_text_cols(data_path, col_name_list3, dimension=70)
    tfidf_pca3.to_csv('requirement_tfidf_pca.csv')

    tfidf_pca4 = get_tfidf_pca_from_text_cols(data_path, col_name_list4, dimension=40)
    tfidf_pca4.to_csv('position_tfidf_pca.csv')

    tfidf_pca5 = get_tfidf_pca_from_text_cols(data_path, col_name_list5, dimension=30)
    tfidf_pca5.to_csv('skills_tfidf_pca.csv')
    
    print("all is well")

'''
jd可以做3个向量
title + category_name + tags        #3607  #25.9s #43.4s
description                         #16284 #4m14.1s
requirement                         #14889 #3m4.8s

cv可以做4个向量：
currentPosition + desiredPosition   #6867 #34.2 #42.5
skills                              #3009 #40.1s #54.2s
jobTracks                           
'''

running...

从文本列获取tfidf_pca向量



/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_52181/667190469.py:19: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built successfully.



==================================title==================================
                 title                      title_jieba title_jieba_filter
0              市场营销负责人                      [市场营销, 负责人]           市场营销 负责人
1               会员管理经理                     [会员, 管理, 经理]           会员 管理 经理
2               产品市场经理                     [产品, 市场, 经理]           产品 市场 经理
3        海外区域运营经理/高级经理      [海外, 区域, 运营, 经理,  , 高级, 经理]  海外 区域 运营 经理 高级 经理
4               高级项目经理                       [高级, 项目经理]            高级 项目经理
...                ...                              ...                ...
81291             技术经理                         [技术, 经理]              技术 经理
81292           中央市场总监                       [中央, 市场总监]            中央 市场总监
81293           中央市场总监                       [中央, 市场总监]            中央 市场总监
81294  java开发工程师（不限方向）  [java, 开发, 工程师, （, 不, 限, 方向, ）]     java 开发 工程师 方向
81295               研发                             [研发]                 研发

[81296 rows x 3 columns]

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_52181/667190469.py:19: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)



==================================description==================================
                                             description  \
0      1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...   
1      1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...   
2      1、•Support regional strategic planning and mar...   
3      1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...   
4      1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...   
...                                                  ...   
81291  职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...   
81292  1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...   
81293  1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...   
81294  1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...   
81295  1、掌握市场动态和产品趋势，对公司产品改进、提升、创新，开发新产品(速冻包点、速冻水饺、预制...   

                                       description_jieba  \
0      [1, 、, 岗位职责, :, 2, 、, 主导, 制定, 公司, 营销, 战略规划, 并,...   
1      [1, 、, 岗位职责, ：, 2, 、, 1, .,  , 负责, “, 九木, 杂物, ...   
2      [1, 、, •, S

...

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_52181/667190469.py:19: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)



==================================currentPosition==================================
      currentPosition currentPosition_jieba currentPosition_jieba_filter
0            副总裁/副总经理        [副总裁,  , 副总经理]                     副总裁 副总经理
1             数字化营销经理         [数字化, 营销, 经理]                    数字化 营销 经理
2                                        []                             
3              泰国国家经理          [泰国, 国家, 经理]                     泰国 国家 经理
4                项目经理                [项目经理]                         项目经理
...               ...                   ...                          ...
81291         生产工、包装工       [生产, 工, 、, 包装工]                       生产 包装工
81292          高级市场总监            [高级, 市场总监]                      高级 市场总监
81293          高级市场总监            [高级, 市场总监]                      高级 市场总监
81294            java                [java]                         java
81295        零售营销部负责人        [零售, 营销部, 负责人]                   零售 营销部 负责人

[81296 rows x 3 columns]

============

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_52181/667190469.py:19: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)



==================================skills==================================
                                                  skills  \
0      ["Ipd","App","Kpi","Crm","View","优化","策划","营销"...   
1      ["H5","C1","Seo","Sem","Top","Kpi","Crm","优化",...   
2      ["Based","Control","Sap Crm","Improve","Indivi...   
3      ["App","Resume","运营","策划","营销","搭建","笔译","舞蹈",...   
4                                               ["智能硬件"]   
...                                                  ...   
81291                                                 电气   
81292    搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写   
81293    搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写   
81294  Bi,Cdn,Elk,Git,Yii,App,Php,Ext,Etl,Java,Sina,H...   
81295  电商,优化,策划,运营,客服,搭建,营销,数据分析,电子商务,运营管理,管理工作,团队建设,...   

                                            skills_jieba  \
0      [Ipd,  , App,  , Kpi,  , Crm,  , View,  , 优化, ...   
1      [H5,  , C1,  , Seo,  , Sem,  , Top,  , Kpi,  ,...   
2      [Based,  , Contr

'\njd可以做3个向量\ntitle + category_name + tags        #3607  #25.9s #43.4s\ndescription                         #16284 #4m14.1s\nrequirement                         #14889 #3m4.8s\n\ncv可以做4个向量：\ncurrentPosition + desiredPosition   #6867 #34.2 #42.5\nskills                              #3009 #40.1s #54.2s\njobTracks                           \n'

In [1]:
import pandas as pd
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_56824/358587841.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
sample = all_data

# 需要将这个向量变成一个列表
def tfidf_pca_merge_fun(series):
    return list(series)

tfidf_pca_file = ['description_tfidf_pca.csv', 'position_tfidf_pca.csv', 'requirement_tfidf_pca.csv', 'skills_tfidf_pca.csv', 'title_category_tags_tfidf_pca.csv']
for csv_file in tfidf_pca_file:
    tfidf_pca = pd.read_csv(csv_file).drop(['Unnamed: 0'], axis=1)
    print(csv_file)
    #print(tfidf_pca)
    col_name = csv_file.strip('.csv')
    tfidf_pca[col_name] = tfidf_pca.apply(tfidf_pca_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_pca[[col_name]]], axis=1)

tfidf_pca_col_names = [csv_file.strip('.csv') for csv_file in tfidf_pca_file]
sample[tfidf_pca_col_names]

description_tfidf_pca.csv
position_tfidf_pca.csv
requirement_tfidf_pca.csv
skills_tfidf_pca.csv
title_category_tags_tfidf_pca.csv


,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca
0,"[-0.2036062295725057, 0.0482743259611426, 0.06...","[-0.0393278199932224, -0.1156517780600516, -0....","[-0.0884550054331397, -0.125519552436948, -0.0...","[0.3299248867367418, 0.164741969808482, 0.1889...","[-0.0428003378972316, -0.1584212552141981, -0...."
1,"[-0.0922193627558096, 0.0553739463988321, -0.1...","[0.1054180773106984, -0.0263971997890037, -0.0...","[-0.0803609394513418, -0.1239125350185267, -0....","[0.2365059699800738, 0.1136815262526436, -0.23...","[-0.2368437904582563, 0.0858237305581985, -0.2..."
2,"[-0.0731212086793756, -0.1819505396457883, -0....","[-0.0368150744980858, -0.0868620766763974, -0....","[-0.0644952418134994, -0.1617720130418969, -0....","[-0.0915394250400994, -0.1409954987383418, -0....","[-0.1908741398655453, 0.1941297757385749, 0.34..."
3,"[-0.0941188808553261, 0.0989578513909629, -0.0...","[0.0490419846368333, -0.0412213764571144, -0.0...","[-0.1351532040885177, 0.1740611212306139, 0.04...","[0.1429876436797454, 0.0219408706921027, -0.14...","[-0.3720983915852013, 0.191217992346987, -0.31..."
4,"[0.0394442348392252, -0.0330071666173531, 0.06...","[-0.0271020068507462, -0.1132012422332272, -0....","[-0.0138174200364112, 0.1003272366963715, -0.0...","[-0.0992961440476657, -0.1495964296220377, -0....","[0.0056217778664832, -0.0571649606124799, 0.00..."
...,...,...,...,...,...
81291,"[-0.0614540249096187, -0.0867970356720048, -0....","[-0.0745961609163453, -0.052911867138294, -0.0...","[-0.0400398533506595, -0.0616278716402495, 0.0...","[-0.0986273352712469, -0.1820012999649072, -0....","[0.0344861435113101, -0.0468051330650081, 0.05..."
81292,"[-0.1197626313820459, 0.2009878509549596, 0.10...","[-0.0248770950143107, -0.0818758151299455, -0....","[-0.1178366614102289, 0.125663028721799, 0.141...","[0.0156513997900681, 0.0238285941465905, -0.06...","[0.0151640498218406, -0.0802995831055218, 0.00..."
81293,"[-0.1197626313820459, 0.2009878509549596, 0.10...","[-0.0248770950143107, -0.0818758151299455, -0....","[-0.1178366614102289, 0.125663028721799, 0.141...","[0.0156513997900681, 0.0238285941465905, -0.06...","[-0.0053327153389531, -0.0845578727706244, 0.0..."
81294,"[0.4074295021733307, -0.0564815548542076, 0.38...","[-0.3849339451860117, 0.4677351578570149, 0.34...","[0.533978782672698, 0.0783524240538382, 0.1293...","[-0.2529650764970855, 0.1589143672355407, 0.08...","[0.547197635656256, 0.4049100240667704, -0.158..."


In [5]:
sample.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

## 数据集划分，剔除复推集